# Denver Crime Heatmap Notebook
CS 3120 Machine Learning Term Project - Konstantin Zaremski

---

This comprehensive notebook covers my project from ideation and planning, to exploratory data analysis (EDA), and final implementation.
### *********** Unfinished Draft (for peer review)

## Exploratory Data Analysis

### Setup
Importing required modules and libraries for EDA, etc. See `requirements_eda.txt` for a full list of requirements to install into the environment using `pip`.

In [11]:
from IPython.display import display
import pandas as pd
import pyproj
import numpy as np
import glob

### Load Crime Data

In [12]:
# Find all crime data CSV files matching the naming pattern
csv_files = glob.glob("data/crime_split_*.csv")

# Read in all of the files and create Pandas dataframes from them
split_dfs = [pd.read_csv(f) for f in csv_files]

# Concatenate all the dataframes into a single one
df = pd.concat(split_dfs, ignore_index=True)

display(df.head())

original_dataframe_shape = df.shape
print(f'Loaded {original_dataframe_shape[0]} records!')

,OBJECTID,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT,x,y
0,20000,2020467360,2020467360299901,2999,1,criminal-mischief-mtr-veh,public-disorder,8/2/2020 10:43:00 PM,NaN,8/2/2020 10:43:00 PM,...,-105.024597,39.689751,4,422,ruby-hill,1,0,1,3.133789e+06,1.676471e+06
1,20001,20196003434,20196003434299901,2999,1,criminal-mischief-mtr-veh,public-disorder,4/20/2019 7:30:00 AM,4/20/2019 8:45:00 AM,4/20/2019 6:30:00 PM,...,-104.987348,39.714316,3,311,speer,1,0,1,3.144221e+06,1.685476e+06
2,20002,2020123049,2020123049299901,2999,1,criminal-mischief-mtr-veh,public-disorder,2/25/2020 11:30:00 PM,2/26/2020 6:45:00 AM,2/26/2020 7:30:00 AM,...,-104.988098,39.764528,6,612,five-points,1,0,1,3.143907e+06,1.703765e+06
3,20003,2021621685,2021621685299901,2999,1,criminal-mischief-mtr-veh,public-disorder,11/1/2021 6:20:00 AM,11/1/2021 6:30:00 AM,11/1/2021 10:15:00 AM,...,-105.004665,39.739669,1,123,lincoln-park,1,0,1,3.139299e+06,1.694684e+06
4,20004,2020289138,2020289138299901,2999,1,criminal-mischief-mtr-veh,public-disorder,5/9/2020 12:00:00 PM,NaN,5/11/2020 12:05:00 PM,...,-104.830661,39.795281,5,521,montbello,1,0,1,3.188083e+06,1.715255e+06


Loaded 394736 records!


### Column Analysis
The next thing to do is determine which columns will be useful for predictions and which columns need to be discarded.

In [13]:
print("Crime Dataset Columns:")
for column in df.columns:
    print(f" - {column}")

Crime Dataset Columns:
 - OBJECTID
 - INCIDENT_ID
 - OFFENSE_ID
 - OFFENSE_CODE
 - OFFENSE_CODE_EXTENSION
 - OFFENSE_TYPE_ID
 - OFFENSE_CATEGORY_ID
 - FIRST_OCCURRENCE_DATE
 - LAST_OCCURRENCE_DATE
 - REPORTED_DATE
 - INCIDENT_ADDRESS
 - GEO_X
 - GEO_Y
 - GEO_LON
 - GEO_LAT
 - DISTRICT_ID
 - PRECINCT_ID
 - NEIGHBORHOOD_ID
 - IS_CRIME
 - IS_TRAFFIC
 - VICTIM_COUNT
 - x
 - y


#### Column Decisions
| Column ID                 | Decison   | Explanation                                                                                        |
|---------------------------|-----------|----------------------------------------------------------------------------------------------------|
| `OBJECTID`                | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `INCIDENT_ID`             | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_ID`              | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_CODE`            | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_CODE_EXTENSION`  | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_TYPE_ID`         | Kept      | This is part of the output (y-values). We will keep this column and it will be the subject of prediction by the classification model. |
| `OFFENSE_CATEGORY_ID`     | Discarded | This is part of the output (y-values). This will be discarded in  |
| `FIRST_OCCURRENCE_DATE`   | Kept      | Timestamp of the event, this will be kept and used for predictions made based on the time of day. |
| `LAST_OCCURRENCE_DATE`    | Discarded | Undefined for many rows. We will discard this timestamp and rely on `FIRST_OCCURRENCE_DATE` instead.                              |
| `REPORTED_DATE`           | Discarded | This is part of the output (y-values). Report time delta from occurrence varies incident to incident. No predictive value.                             |
| `INCIDENT_ADDRESS`        | Discarded | Although address is useful, this column is not standardized with entries like "2400 block" rather than exact addresses in some cases. |
| `GEO_X`                   | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |
| `GEO_Y`                   | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |
| `GEO_LON`                 | Kept      | Location of the crime, this will be kept and used to make predictions based on location. |
| `GEO_LAT`                 | Kept      | Location of the crime, this will be kept and used to make predictions based on location. |
| `DISTRICT_ID`             | Discarded | This is part of the output (y-values) and we have no way of querying. No predictive value.                    |
| `PRECINCT_ID`             | Discarded | This is part of the output (y-values) and we have no way of querying. No predictive value.                             |
| `NEIGHBORHOOD_ID`         | Discarded | Although neighborhood is useful, as some are "rougher" than others, we don't have a standard way of querying.                             |
| `IS_CRIME`                | Discarded | This is part of the output (y-values). No predictive value.                             |
| `IS_TRAFFIC`              | Discarded | This is part of the output (y-values). No predictive value.                             |
| `VICTIM_COUNT`            | Discarded | This is part of the output (y-values). No predictive value.                             |
| `x`                       | Discarded | Related to the mapping platform the data was posted on. No predictive value.                     |
| `y`                       | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |

In [14]:
# Keep only the columns that we want
df = df[['OFFENSE_TYPE_ID','FIRST_OCCURRENCE_DATE','GEO_LON','GEO_LAT']]

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=20))

,OFFENSE_TYPE_ID,FIRST_OCCURRENCE_DATE,GEO_LON,GEO_LAT
249312,criminal-mischief-other,2/15/2024 1:40:00 PM,-104.977182,39.740618
152995,burglary-residence-no-force,6/4/2021 6:30:00 AM,-105.027892,39.711870
7782,criminal-mischief-graffiti,5/4/2021 7:15:00 PM,-104.910718,39.714221
56694,theft-items-from-vehicle,8/20/2021 5:30:00 PM,-104.992332,39.752217
47529,burglary-residence-by-force,6/24/2023 5:30:00 AM,-104.986670,39.677293
20815,theft-items-from-vehicle,9/23/2021 7:00:00 PM,-104.909615,39.712991
196339,theft-of-motor-vehicle,6/10/2021 10:25:00 AM,-104.985392,39.674723
385248,theft-of-motor-vehicle,2/7/2023 8:15:00 PM,-104.954866,39.742230
224798,theft-of-motor-vehicle,6/26/2021 7:24:00 AM,-104.967325,39.665641
338529,theft-unauth-use-of-ftd,10/21/2021 8:50:00 AM,-104.924566,39.701315


### Time Binning
In this analysis, time binning is used to capture the cyclical and seasonal patterns inherent in criminal behavior. Crime often follows temporal trends—certain types of incidents may be more likely to occur at specific times of day, days of the week, or during particular seasons. For example, incidents related to nightlife might peak during late evening hours, while other crimes may correlate with daily commuter traffic or specific days like weekends. By extracting temporal features like `YEAR`, `DAY_OF_YEAR`, `DAY_OF_WEEK`, and a continuous `TIME` (floating-point hour), we enable the model to identify these patterns and trends. This granular binning enhances the predictive power of the model, allowing it to detect not only broad time-based patterns but also more subtle nuances in how crime evolves throughout the day, week, and year.

#### Temporal Features Breakdown
* `YEAR`: Provides a clear yearly context, which can be crucial for identifying long-term trends or annual shifts in patterns.
* `TIME` (floating point): Offers a precise measure of the time of day, allowing models to pick up on subtle temporal shifts within a 24-hour cycle.
* `DAY_OF_YEAR`: Captures the day within the year, useful for identifying seasonal patterns or trends that recur annually.
* `DAY_OF_WEEK`: Encodes weekly cycles, helping the model detect patterns associated with specific weekdays.

In [15]:
# Convert FIRST_OCCURRENCE_DATE to Pandas date time format
df['FIRST_OCCURRENCE_DATE'] = pd.to_datetime(df['FIRST_OCCURRENCE_DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract date and time components
df['YEAR'] = df['FIRST_OCCURRENCE_DATE'].dt.year
# df['MONTH'] = df['FIRST_OCCURRENCE_DATE'].dt.month -- Removed in favor of DAY_OF_YEAR
# df['DAY_OF_MONTH'] = df['FIRST_OCCURRENCE_DATE'].dt.day -- Removed in favor of DAY_OF_YEAR
# df['HOUR'] = df['FIRST_OCCURRENCE_DATE'].dt.hour -- Removed in favor of a floating point hour describing the time of day with more precision
df['TIME'] = (
    df['FIRST_OCCURRENCE_DATE'].dt.hour + 
    df['FIRST_OCCURRENCE_DATE'].dt.minute / 60 + 
    df['FIRST_OCCURRENCE_DATE'].dt.second / 3600
)
df['DAY_OF_YEAR'] = df['FIRST_OCCURRENCE_DATE'].dt.dayofyear
df['DAY_OF_WEEK'] = df['FIRST_OCCURRENCE_DATE'].dt.dayofweek  # 0 = Monday, 6 = Sunday

# Finally, dropping the FIRST_OCCURRENCE_DATE column
df.drop(columns=['FIRST_OCCURRENCE_DATE'], inplace=True)

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=5))

,OFFENSE_TYPE_ID,GEO_LON,GEO_LAT,YEAR,TIME,DAY_OF_YEAR,DAY_OF_WEEK
313951,arson-business,-104.899608,39.696534,2022,22.116667,347,1
49306,theft-of-motor-vehicle,-104.704423,39.830901,2023,9.000000,167,4
213426,prostitution-engaging-in,-104.897559,39.740158,2024,12.383333,227,2
143101,public-fighting,-104.999951,39.753054,2020,8.483333,26,6
65555,criminal-mischief-mtr-veh,-104.989865,39.736265,2021,12.166667,171,6


### Location Slicing & Binning 
In this step, we convert latitude and longitude into a 15-meter by 15-meter grid system using a projected coordinate system (UTM). This method ensures that each crime location is assigned to a unique, precise grid cell, facilitating accurate spatial analysis.

See <https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system>

#### Why This is Better Than Using Raw Latitude and Longitude

Using UTM coordinates overcomes the limitations of raw latitude and longitude, which vary in distance depending on location. UTM coordinates provide uniform distance measurements in meters, improving precision and making it easier to define consistent grid cells. This approach also simplifies spatial analysis by grouping data into fixed-size cells, making patterns like crime hotspots easier to identify and computationally more efficient to process.

In [16]:
# Initialize UTM projection (assuming UTM Zone 13N for Denver)
#   https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system
#   https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#/media/File:Universal_Transverse_Mercator_zones.svg
utm_proj = pyproj.Proj(proj="utm", zone=13, datum="WGS84")

def get_grid_block(lat, lon, grid_size=15):
    try:
        # Convert latitude and longitude to UTM coordinates (X, Y)
        x, y = utm_proj(lon, lat)  # Correct order: lon, lat
        
        # Calculate the X and Y blocks
        x_block = int(x // grid_size)
        y_block = int(y // grid_size)
        
        return x_block, y_block
    except:
        # If there is an issue parsing, return -1
        return -1, -1

# Apply the function to the DataFrame
df[['X_BLOCK', 'Y_BLOCK']] = df.apply(lambda row: get_grid_block(row['GEO_LAT'], row['GEO_LON']), axis=1, result_type='expand')

# Drop rows where X_BLOCK or Y_BLOCK is -1
df = df[(df['X_BLOCK'] != -1) & (df['Y_BLOCK'] != -1)]

# Note change in size
post_null_geo_block_removal_dataframe_shape = df.shape
print(f'Dataframe size before removal: {original_dataframe_shape[0]} items')
print(f'Dataframe size after removal: {post_null_geo_block_removal_dataframe_shape[0]} items')
print(f'--> {original_dataframe_shape[0] - post_null_geo_block_removal_dataframe_shape[0]} items removed!')

# Finally, dropping the GEO_LAT and GEO_LON columns since we have replaced them with X_BLOCK and Y_BLOCK
df.drop(columns=['GEO_LAT'], inplace=True)
df.drop(columns=['GEO_LON'], inplace=True)

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=5))

Dataframe size before removal: 394736 items
Dataframe size after removal: 394475 items
--> 261 items removed!


,OFFENSE_TYPE_ID,YEAR,TIME,DAY_OF_YEAR,DAY_OF_WEEK,X_BLOCK,Y_BLOCK
117720,violation-of-court-order,2020,15.866667,218,2,33186,292838
327482,theft-other,2020,12.116667,332,4,33128,292675
190952,criminal-trespassing,2021,1.350000,352,5,33333,293356
139225,weapon-unlawful-discharge-of,2020,12.000000,166,6,33185,292907
58141,theft-items-from-vehicle,2021,23.000000,63,3,33645,293225


In this section, we are processing the geographical data to assign each crime incident to a specific grid cell based on its latitude and longitude. We achieve this by converting the coordinates to UTM projection, which provides more accurate distance-based calculations than traditional latitude and longitude. For each point, we calculate the corresponding grid cell by dividing the UTM coordinates by the grid size (15 meters) and then storing the results as `X_BLOCK` and `Y_BLOCK`.

However, if any coordinates cannot be processed correctly (due to projection issues or invalid data), we assign -1 as a placeholder. After applying this transformation, we remove any rows where either the `X_BLOCK` or `Y_BLOCK` is -1, indicating invalid grid assignments. The sizes of the DataFrame before and after this removal are printed to highlight the impact of cleaning the data. This step ensures that only valid data points are included for further analysis and modeling.

### Adding in OpenStreetMap Features